In [13]:
import os
import struct
import numpy as np
import scipy.io as si
import pandas as pd
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
from torchvision import transforms, datasets
import time
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from CenterLoss import CenterLoss
import matplotlib.pyplot as plt

class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        
        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        img = img.astype( np.float32 )
        # h and w are swapped for landmarks because for images,
        # x and y axes are axis 1 and 0 respectively

        return {'image': img, 'label': label}

In [3]:
class ClothesDataset(Dataset):
    """clothes dataset."""
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.clothes_label = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.clothes_label)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.clothes_label.iloc[idx, 0])
        image = io.imread(img_name)
        image = image.astype( np.float32 )
        label = self.clothes_label.iloc[ idx, 1]
        sample = {'image':image, 'label':label }

        if self.transform:
            sample = self.transform(sample)

        return sample
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'label': label }

In [64]:
class Net(nn.Module):
    def __init__(self, num = 24 ):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, stride = 2, padding = [2,2]) #10 * 24 * 24
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5, stride = 2, padding = [2,2] ) #20 * 12 * 12
        self.conv3 = nn.Conv2d(64, 64, kernel_size=5, stride = 2, padding = [2,2] ) #20 * 12 * 12
        
        
        self.res1 = nn.Conv2d( 64, 64, kernel_size=5,padding=[2,2] )
        self.res2 = nn.Conv2d( 64, 64, kernel_size=5,padding=[2,2] )
        self.res3 = nn.Conv2d( 64, 64, kernel_size=5,padding=[2,2] )
        
        self.bn1 = nn.BatchNorm2d( 64 )
        self.bn2 = nn.BatchNorm2d( 64 )
        self.bn3 = nn.BatchNorm2d( 64 )
        
        
        self.m1 = nn.MaxPool2d(2, stride=2)
        self.m2 = nn.MaxPool2d(2, stride=2)
        self.m3 = nn.MaxPool2d(2, stride=2)
        
        self.fc1 = nn.Linear(64 * (num // 64 ) * ( num // 64 ), 128)
        self.fc1_BN = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 8)
        self.fc3 = nn.Linear( 8, 8)
#         self.fc4 = nn.Linear( 2, 10 )
        self.num = num
        
        
    def forward(self, x):
        x = F.relu( self.conv1( x ) )
        x = F.relu( self.res1( x ) ) +  x
        x = self.bn1( x )
        x = self.m1( x )
        
        x = F.relu( self.conv2( x ) )
        x = F.relu( self.res2( x ) ) +  x
        x = self.bn2( x )
        x = self.m2( x )
        
        x = F.relu( self.conv3( x ) )
        x = F.relu( self.res3( x ) ) +  x
        x = self.bn3( x )
        x = self.m3( x )
        
        x = x.view(-1, 64 * (self.num // 64 ) * (self.num // 64 ) )
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc1_BN( x )
        x = F.dropout(x, training=self.training) 
        ip1 = self.fc2(x) #10
        ip2 = self.fc3(ip1)
        
        return ip1, F.log_softmax(ip2, dim=1)

In [65]:
train_csv_file = 'train.csv'
test_csv_file = 'val.csv'
root_dir = 'data/image/pro_train'

composed = transforms.Compose([Rescale((128,128)),ToTensor()] )
transformed_train_data = ClothesDataset(  train_csv_file,root_dir, composed )
transformed_test_data = ClothesDataset( test_csv_file,root_dir, composed )

train_loader = torch.utils.data.DataLoader( transformed_train_data, shuffle=True, batch_size=100)
test_loader = torch.utils.data.DataLoader( transformed_test_data, shuffle=True, batch_size=100)

In [ ]:
num = 128
torch.manual_seed(0)
device = torch.device("cpu")
model = Net(num=num).to(device)

nllloss = nn.NLLLoss().to(device) #CrossEntropyLoss = log_softmax + NLLLoss
# CenterLoss
loss_weight = 0.001
feat = 10
centerloss = CenterLoss(8, 8).to(device)

lr = 0.001
for epoch in range(100):
    
    if lr < 1e-6:
        lr = 1e-6
    
    optimizer4nn = optim.Adam(model.parameters(), lr=lr)
    optimzer4center = optim.Adam(centerloss.parameters(), lr = lr )
    # print optimizer4nn.param_groups[0]['lr']
    train(epoch+1, train_loader=train_loader, log_interval=10)
    test( model, device, test_loader )
    
    lr = lr * 0.98

Training... Epoch = 1


/Users/chengzhilin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/chengzhilin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Train Epoch: 1 [0/16838 (0%)]	Loss: 8.022339
Train Epoch: 1 [1000/16838 (6%)]	Loss: 7.194895
Train Epoch: 1 [2000/16838 (12%)]	Loss: 7.119150
Train Epoch: 1 [3000/16838 (18%)]	Loss: 6.899786
Train Epoch: 1 [4000/16838 (24%)]	Loss: 6.568599
Train Epoch: 1 [5000/16838 (30%)]	Loss: 6.294324
Train Epoch: 1 [6000/16838 (36%)]	Loss: 6.097884
Train Epoch: 1 [7000/16838 (41%)]	Loss: 5.947513
Train Epoch: 1 [8000/16838 (47%)]	Loss: 5.787264
Train Epoch: 1 [9000/16838 (53%)]	Loss: 5.535451
Train Epoch: 1 [10000/16838 (59%)]	Loss: 5.483101


In [71]:
class Net(nn.Module):
    def __init__(self, num = 24 ):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, stride = 2, padding = [2,2]) #10 * 24 * 24
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5, stride = 2, padding = [2,2] ) #20 * 12 * 12
        self.conv3 = nn.Conv2d(64, 64, kernel_size=5, stride = 2, padding = [2,2] ) #20 * 12 * 12
        
        
        self.res1 = nn.Conv2d( 64, 64, kernel_size=5,padding=[2,2] )
        self.res2 = nn.Conv2d( 64, 64, kernel_size=5,padding=[2,2] )
        self.res3 = nn.Conv2d( 64, 64, kernel_size=5,padding=[2,2] )
        
        self.bn1 = nn.BatchNorm2d( 64 )
        self.bn2 = nn.BatchNorm2d( 64 )
        self.bn3 = nn.BatchNorm2d( 64 )
        
        
        self.m1 = nn.MaxPool2d(2, stride=2)
        self.m2 = nn.MaxPool2d(2, stride=2)
        self.m3 = nn.MaxPool2d(2, stride=2)
        
        self.fc1 = nn.Linear(64 * (num // 64 ) * ( num // 64 ), 128)
        self.fc1_BN = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 8)
        self.fc3 = nn.Linear( 8, 8)
#         self.fc4 = nn.Linear( 2, 10 )
        self.num = num
        
        
    def forward(self, x):
        x = F.relu( self.conv1( x ) )
        x = F.relu( self.res1( x ) ) +  x
        x = self.bn1( x )
        x = self.m1( x )
        
        x = F.relu( self.conv2( x ) )
        x = F.relu( self.res2( x ) ) +  x
        x = self.bn2( x )
        x = self.m2( x )
        
        x = F.relu( self.conv3( x ) )
        x = F.relu( self.res3( x ) ) +  x
        x = self.bn3( x )
        x = self.m3( x )
        
        x = x.view(-1, 64 * (self.num // 64 ) * (self.num // 64 ) )
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc1_BN( x )
        x = F.dropout(x, training=self.training) 
        ip1 = self.fc2(x) #10
        ip2 = self.fc3(ip1)
        
        return ip1, F.log_softmax(ip2, dim=1)